In [3]:
from pyspark.ml.classification import LinearSVC
from pyspark import SparkContext
from pyspark import SparkContext
from pyspark.sql import SQLContext

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("Pistachio_16_Features_Dataset.csv")
df=df.dropna()
df['AREA']=df['AREA'].astype(int)
from sklearn.preprocessing import LabelEncoder
lc=LabelEncoder()
df['Class']=lc.fit_transform(df['Class'])
df

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('KMeans').getOrCreate()

In [ ]:
data1 = spark.createDataFrame(df)

In [ ]:
data1

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
cl=list(df.columns[0:16])

In [ ]:
assembler = VectorAssembler(inputCols=cl,outputCol="features")

In [ ]:
d=assembler.transform(data1)
d = d.select(['features','Class'])
d.show()

In [ ]:
type(d)

In [ ]:
train,test=d.randomSplit([0.8,0.2],seed=42)

In [ ]:
from pyspark.ml.feature import StandardScaler
  
scaler = StandardScaler(inputCol="features", 
                        outputCol="scaledFeatures", 
                        withStd=True, 
                        withMean=False)
scalerModel = scaler.fit(d)
d= scalerModel.transform(d)
  
d.select('scaledFeatures').show(5)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
  
silhouette_score=[]
  
evaluator = ClusteringEvaluator(predictionCol='prediction',
                                featuresCol='scaledFeatures', \
                                metricName='silhouette', 
                                distanceMeasure='squaredEuclidean')
  
for i in range(2,10):
    kmeans=KMeans(featuresCol='scaledFeatures', k=i)
    model=kmeans.fit(d)
    predictions=model.transform(d)
    score=evaluator.evaluate(predictions)
    silhouette_score.append(score)
    print('Silhouette Score for k =',i,'is',score)

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(2,10),silhouette_score)
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.title('Silhouette Score')
plt.show()

In [4]:
!pip install pyspark

Unable to create process using 'C:\Users\Naboth Demetrius\anaconda3\python.exe "C:\Users\Naboth Demetrius\anaconda3\Scripts\pip-script.py" install pyspark'
